In [56]:
# Import necessary libraries
import yfinance as yf 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

In [57]:
# Fetch historical data for S&P 500
sp500 = yf.Ticker("^GSPC")
sp500 = sp500.history(period="max")

In [58]:
# Drop unnecessary columns
del sp500["Stock Splits"]
del sp500["Dividends"]

In [59]:
# Create target variable: whether the open price of the next day is higher than the close price of today
sp500["Open_Tomorrow"] = sp500["Open"].shift(-1)
sp500["Target"] = (sp500["Open_Tomorrow"] > sp500["Close"]).astype(int)

In [60]:
# Select data from 1990 onwards
sp500 = sp500.loc["1990-01-01":].copy()

In [61]:
# Initialize and train a RandomForest model
model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1)



In [62]:
# Define training and testing datasets
train = sp500.iloc[:-100]
test = sp500.iloc[-100:]

In [63]:
# Define predictors
predictors = ["Close", "Volume", "Open", "High", "Low"]

In [64]:
# Fit the model on the training data
model.fit(train[predictors], train["Target"])

RandomForestClassifier(min_samples_split=100, random_state=1)

In [65]:
# Make predictions on the test set
preds = model.predict(test[predictors])

In [66]:
# Convert predictions to a pandas Series for ease of analysis
preds = pd.Series(preds, index=test.index)

In [67]:
# Evaluate initial model performance
precision_score(test["Target"], preds)

0.67

In [68]:
# Combine actual and predicted values for further analysis
combined = pd.concat([test["Target"], preds], axis=1)

In [69]:
# Function to fit model and make predictions
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [70]:
# Function to perform backtesting
def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []
    
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    return pd.concat(all_predictions)
    

In [71]:
# Perform backtesting
predictions = backtest(sp500, model, predictors)

In [72]:
# Analyze predictions
predictions["Predictions"].value_counts()

0    4235
1    1993
Name: Predictions, dtype: int64

In [73]:
precision_score(predictions["Target"], predictions["Predictions"])

0.5594581033617662

In [74]:
# Define horizons for feature engineering
horizons = [2,5,60,250,1000]
new_predictors = []

for horizon in horizons:
    rolling_averages = sp500.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    sp500[ratio_column] = sp500["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]
    
    new_predictors += [ratio_column, trend_column]

new_predictors += predictors

In [75]:
# Drop rows with NaN values resulting from feature engineering
sp500 = sp500.dropna()

In [76]:
# Initialize and train a new RandomForest model with updated parameters
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

In [77]:
# Perform backtesting with new features
predictions = backtest(sp500, model, new_predictors)

In [78]:
# Analyze predictions following new features
predictions["Predictions"].value_counts()

0    3508
1    1719
Name: Predictions, dtype: int64

In [79]:
precision_score(predictions["Target"], predictions["Predictions"])

0.5532286212914486

In [80]:
# Refine further with ensemble approach

In [81]:
# Define multiple models to start ensemble approach
xgb_model = XGBClassifier()
rf_model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

In [82]:
# Ensemble model
ensemble_model = VotingClassifier(estimators=[('rf', rf_model), ('xgb', xgb_model)], voting='soft')

In [83]:
# Train and predict using the ensemble model
ensemble_model.fit(train[predictors], train["Target"])
ensemble_preds = ensemble_model.predict_proba(test[predictors])[:, 1]

In [84]:
# Convert probabilities to binary predictions
ensemble_preds[ensemble_preds >= 0.6] = 1
ensemble_preds[ensemble_preds < 0.6] = 0

In [85]:
# Evaluate ensemble model
ensemble_precision = precision_score(test["Target"], ensemble_preds)
print("Ensemble Precision Score:", ensemble_precision)

Ensemble Precision Score: 0.6481481481481481


In [86]:
# Prepare the latest available data
latest_data = sp500.iloc[-1:]  # Get the most recent row of data

In [87]:
# Calculate new features for the latest data
latest_data = latest_data.copy()

for horizon in horizons:
    rolling_averages = sp500.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    trend_column = f"Trend_{horizon}"
    
    # Calculate the ratio and trend
    latest_data[ratio_column] = latest_data["Close"] / rolling_averages["Close"].iloc[-1]
    latest_data[trend_column] = latest_data.shift(1).rolling(horizon).sum()["Target"].iloc[-1]

In [88]:
# Ensure the latest_data has all predictors
latest_data = latest_data[predictors]

In [89]:
# Use the trained ensemble model to make predictions
latest_preds = ensemble_model.predict_proba(latest_data)[:, 1]  # Probability of the positive class
latest_preds_binary = (latest_preds >= 0.6).astype(int)  # Convert to binary prediction

In [90]:
f"Predicted probability for the next day: {latest_preds[0]}"

'Predicted probability for the next day: 0.636097538650341'

In [91]:
f"Binary prediction (1 for up, 0 for down): {latest_preds_binary[0]}"

'Binary prediction (1 for up, 0 for down): 1'